CREATING GENERATED AND REFERENCE SENTENTENCE

In [ ]:
import string
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.model_selection import train_test_split
import csv
import re
import pandas as pd
import pickle
import torch
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
user_input_list = []
response_list = []
keyword_list = []

def remove_start_integer(utterance):
    match = re.match(r'^\d+', utterance)
    if match:
        leading_integer = match.group()
        return utterance[len(leading_integer):].lstrip()
    return utterance

with open('/content/drive/MyDrive/BTech_Project/keywords.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)

    rows = list(csv_reader)

    for i in range(len(rows)):
        if(i+1 == len(rows)):
            break
        if(rows[i][0] == "" or rows[i+1][0] == ""):
            continue
        user_input = remove_start_integer(rows[i][0])
        response = remove_start_integer(rows[i+1][0])
        keyword = rows[i+1][3]

        response_list.append(response)
        keyword_list.append(keyword)
        # user_input_list.append(f"{user_input} KW {keyword}")

        hist_list = []
        hist_list.append(rows[i][0] + " KW " + keyword)
        for j in range(i-1, i-3, -1):
            if (j < 0 or rows[j][0] == ""):
                break
            hist_list.append(rows[j][0])

        conv_hist = ""
        ct = 0

        for dialogue in reversed(hist_list):
            if ct == 0:
                conv_hist = dialogue
            else:
                conv_hist = conv_hist + tokenizer.eos_token + dialogue
            ct += 1

        user_input_list.append(conv_hist)

df = pd.DataFrame({'Response': response_list, 'Input': user_input_list, 'Keywords':keyword_list})
df.head(20)

,Response,Input,Keywords
0,you must be very fast . hunting is one of my f...,"hi , how are you doing ? i am getting ready to...",hunt
1,i am ! for my hobby i like to do canning or so...,"hi , how are you doing ? i am getting ready to...",can
2,i also remodel homes when i am not out bow hun...,"hi , how are you doing ? i am getting ready to...",home
3,that is neat . when i was in high school i pla...,you must be very fast . hunting is one of my f...,dash
4,that is awesome . do you have a favorite seaso...,i am ! for my hobby i like to do canning or so...,season
5,i do not . but i do have a favorite meat since...,i also remodel homes when i am not out bow hun...,meat
6,what is your favorite meat to eat ?,that is neat . when i was in high school i pla...,meat
7,i would have to say its prime rib . do you hav...,that is awesome . do you have a favorite seaso...,rib
8,i like chicken or macaroni and cheese .,i do not . but i do have a favorite meat since...,macaroni
9,do you have anything planned for today ? i thi...,what is your favorite meat to eat ?<|endoftext...,can


In [ ]:
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/BTech_Project/fine_tune_dialoGPT_ctx_hist5_1epochs_random_state42')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/BTech_Project/fine_tune_dialoGPT_ctx_hist5_1epochs_random_state42')

og_model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')
og_tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')

In [ ]:
trn_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
trn_df, test_df = train_test_split(trn_df, test_size=0.005, random_state=42)
print('Length of Training Data: ', len(trn_df))
print('Length of Val Data: ', len(val_df))
print('Length of testing Data: ', len(test_df))


Length of Training Data:  109697
Length of Val Data:  12250
Length of testing Data:  552


In [ ]:
# test_df
# eos_count = test_df['Input'].str.count("<\|endoftext\|>")
# test_df = test_df[eos_count == 1]
# filtered_df
# len(test_df)

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

In [ ]:
def remove_punctuation(sentence):
    filtered_sentence = ''.join(char for char in sentence if char not in string.punctuation)
    return filtered_sentence

# reference_sentences = []
ft_generated_sentences = []
# keyword_list = []
# og_generated_sentences = []

for index, row in test_df.iterrows():
    # response = row['Response']
    input = row['Input']
    # keyword = row['Keywords']

    new_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')


    chat_history_ids = model.generate(
        new_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        # no_repeat_ngram_size=3,
        # do_sample=True,
        # top_k=100,
        # top_p=0.7,
        # temperature = 0.8,
                                        )

    reply = tokenizer.decode(chat_history_ids[0]).split(tokenizer.eos_token)
    reply = reply[len(reply)-2]

    # filtered_response = remove_punctuation(response)
    filtered_reply =  remove_punctuation(reply)

    # reference_sentences.append(filtered_response)
    ft_generated_sentences.append(filtered_reply)
    # keyword_list.append(keyword)

    # original dialogpt

    # input = input.split("KW")[0]
    # print(input)
    # new_user_input_ids = og_tokenizer.encode(input + og_tokenizer.eos_token, return_tensors='pt')

    # chat_history_ids = og_model.generate(new_user_input_ids, max_length=1000, pad_token_id=og_tokenizer.eos_token_id)

    # og_reply = og_tokenizer.decode(chat_history_ids[:, new_user_input_ids.shape[-1]:][0], skip_special_tokens=True)
    # og_filtered_reply =  remove_punctuation(og_reply)

    # og_generated_sentences.append(og_filtered_reply)

    # break

In [ ]:
# len(og_generated_sentences), len(ft_generated_sentences), len(reference_sentences), len(keyword_list)

72

In [ ]:
ftemp = open("/content/drive/MyDrive/BTech_Project/evaluation_test_size_0.005/test_data_epoch2.pkl", "rb")
(og_generated_sentences, temp, reference_sentences, keyword_list) = pickle.load(ftemp)

with open("/content/drive/MyDrive/BTech_Project/evaluation_test_size_0.005/epoch1_ctx_hist5_on_hist3.pkl", "wb") as f:
    pickle.dump((og_generated_sentences, ft_generated_sentences, reference_sentences, keyword_list) , f)

EVALUATION

In [ ]:
f = open("/content/drive/MyDrive/BTech_Project/evaluation_test_size_0.005/test_data_epoch3.pkl", "rb")
(og_generated_sentences, ft_generated_sentences, reference_sentences, keyword_list) = pickle.load(f)

In [ ]:
model_gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
!pip install evaluate
!pip install rouge_score

nltk.download('punkt')
nltk.download('wordnet')

  Using cached rouge_score-0.1.2-py3-none-any.whl


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import evaluate

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def get_bert_embeddings(sentence):
    inputs = bert_tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        hidden_states = outputs.hidden_states
    embeddings = hidden_states[-1]
    sentence_embedding = torch.mean(embeddings, dim=1).squeeze()
    sentence_embedding = sentence_embedding.numpy()
    return sentence_embedding

# def is_word_present(sentence, keyword):
#     tokens = word_tokenize(sentence)
#     print(tokens)
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_keyword = lemmatizer.lemmatize(keyword)
#     for token in tokens:
#         if lemmatizer.lemmatize(token) == lemmatized_keyword:
#             return True
#     return False

from nltk.corpus import wordnet as wn

def is_word_or_similar(sentence, word):

    words = word_tokenize(sentence.lower())

    target_synsets = wn.synsets(word)

    # Check for exact match
    if word in words:
        return True

    # Check for similar words using path similarity
    for w in words:
        w_synsets = wn.synsets(w)
        for ts in target_synsets:
            for ws in w_synsets:
                path_sim = ws.path_similarity(ts)
                if path_sim is not None and path_sim >= 0.9:
                    # print(w, word)
                    return True

    # No match found
    return False

def cosine_similarity_score(embedding1, embedding2):
    similarity = cosine_similarity([embedding1], [embedding2])
    return similarity[0][0]

In [ ]:
# wn.synsets("good"), wn.synsets("goods")

In [ ]:
def calculate_word_accuracy(keywords, generated_sentences):
  total_word_accuracy = 0
  total_n = 0
  for keyword, sentence in zip(keywords, generated_sentences):
    if(is_word_or_similar(sentence,keyword)):
      total_word_accuracy += 1
    total_n += 1
  if total_n == 0:
    return 0
  average_word_accuracy = total_word_accuracy/total_n
  return average_word_accuracy

def calculate_cosine_simscore(reference_sentences, generated_sentences):
  total_cosine_similarity = 0
  total_n = 0
  for reference,generated in zip(reference_sentences, generated_sentences):
    embedding1 = get_bert_embeddings(generated)
    embedding2 = get_bert_embeddings(reference)
    similarity_score = cosine_similarity_score(embedding1, embedding2)
    similarity_score = (similarity_score + 1)/2
    total_cosine_similarity += similarity_score
    total_n += 1
  if total_n == 0:
    return 0
  average_cosine_similarity = total_cosine_similarity / total_n
  return average_cosine_similarity

def calculate_perplexity(generated_sentences):
  total_ppl = 0
  for text in generated_sentences:
    inputs = tokenizer_gpt2(text, return_tensors = "pt")
    loss = model_gpt2(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
    ppl = torch.exp(loss)
    total_ppl += int(ppl)
  if len(generated_sentences) == 0:
    return 0
  return total_ppl/len(generated_sentences)


In [ ]:
print(reference_sentences[0:10])
print(ft_generated_sentences[0:10])
print(keyword_list[0:10])

['i serve lunch  lol  i am a waitress at my parents diner', 'what subjects do you teach ', 'no  too busty exploring my options  pros of being an athlete', 'i do but i fall some broke my arm two different times', 'she is trying to eat my vegetables  and get attention  haha ', 'the perks of being a teacher  my son loves his summer break ', 'cool  i am hoping to teach english in spain', 'that stinks  i love to drive for hours ', 'hey i play drums too  i play rock  country and all kinds', 'cool  i love california it is beautiful']
['i am a waitress and i am learning to cook', 'i bet that is a lot of work', 'i am not i am a pro tennis player', 'i do but i have to walk with my arms extended out to watch the sun come up', 'i am so glad you are not a cat person', 'i agree i am a bit of a loner so summer vacation is nice', 'i like to listen to music while i draw i like spanish music', 'oh no i am sorry i work long hours which can get exhausting', 'i play the drums too i am a drummer in a band',

In [ ]:
word_accuracy = calculate_word_accuracy(keyword_list, ft_generated_sentences)

print("WORD ACCURACY :: ",word_accuracy)
print("")

cosine_score = calculate_cosine_simscore(reference_sentences, ft_generated_sentences)

print("COSINE SIMILARITY :: ",cosine_score)
print("")

print("BLEU SCORE :: ")
references = []
for text in reference_sentences:
  references.append([text])
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=ft_generated_sentences, references=references)

print(results)
print("")

print("ROGUE SCORE ")
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=ft_generated_sentences, references=reference_sentences)
print(results)

print("")

perplexity = calculate_perplexity(ft_generated_sentences)
print("PERPLEXITY SCORE :: ",perplexity)

WORD ACCURACY ::  0.5452898550724637

COSINE SIMILARITY ::  0.8656482910062524

BLEU SCORE :: 
{'bleu': 0.10578895478967358, 'precisions': [0.2931407942238267, 0.126102646351243, 0.07369844489519946, 0.049665465774575396], 'brevity_penalty': 0.9808712423850453, 'length_ratio': 0.9810518859571454, 'translation_length': 5540, 'reference_length': 5647}

ROGUE SCORE 
{'rouge1': 0.308025244824672, 'rouge2': 0.14490959481433435, 'rougeL': 0.2908016006901204, 'rougeLsum': 0.29064466630983105}



ValueError: cannot convert float NaN to integer

In [ ]:
for i in range(len(reference_sentences)):
    print(ft_generated_sentences[i],"||", reference_sentences[i])

i am a waitress and i love reading || i serve lunch  lol  i am a waitress at my parents diner
i have a lot of them too what subject do you teach || what subjects do you teach 
i am not married i have a pro football career || no  too busty exploring my options  pros of being an athlete
yes i do i am a vegan and i love to walk || i do but i fall some broke my arm two different times
i do like attention || she is trying to eat my vegetables  and get attention  haha 
i am more of a day person i do not like the heat || the perks of being a teacher  my son loves his summer break 
oh i see do you have a favorite spanish artist || cool  i am hoping to teach english in spain
i am sorry i love green too i drive a hour to work every day || that stinks  i love to drive for hours 
oh i have a drum set but i am not sure if i could play them || hey i play drums too  i play rock  country and all kinds
i am from californias and love it there || cool  i love california it is beautiful
he does not he is 

In [ ]:
# original dialogpt

word_accuracy = calculate_word_accuracy(keyword_list, og_generated_sentences)

print("WORD ACCURACY :: ",word_accuracy)
print("")

# cosine_score = calculate_cosine_simscore(reference_sentences, og_generated_sentences)

# print("COSINE SIMILARITY :: ",cosine_score)
# print("")

print("BLEU SCORE :: ")
references = []
for text in reference_sentences:
  references.append([text])
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=og_generated_sentences, references=references)

print(results)
print("")

print("ROGUE SCORE ")
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=og_generated_sentences, references=reference_sentences)
print(results)

print("")

perplexity = calculate_perplexity(og_generated_sentences)
print("PERPLEXITY SCORE :: ",perplexity)

WORD ACCURACY ::  0.07971014492753623

BLEU SCORE :: 


{'bleu': 0.012778538816206017, 'precisions': [0.15561486347772532, 0.02546885853206761, 0.00689838153356328, 0.0018444512757454657], 'brevity_penalty': 0.8527317977677189, 'length_ratio': 0.8625819018948114, 'translation_length': 4871, 'reference_length': 5647}

ROGUE SCORE 


{'rouge1': 0.14338704742083402, 'rouge2': 0.02429031549237524, 'rougeL': 0.13478587751243926, 'rougeLsum': 0.13509051531148353}

PERPLEXITY SCORE ::  2293.663043478261


In [ ]:
#remove stopwords?

ref_dash = []
for i in reference_sentences:
    s = word_tokenize(i)
    ref_dash.append(s)

from nltk.translate import meteor_score

meteor_avg_score = meteor_score.meteor_score(ref_dash, ft_generated_sentences)

print("METEOR Score:", meteor_avg_score)

print(reference_sentences)
print(ref_dash)
print(ft_generated_sentences)

TEST


In [ ]:
f = open("/content/drive/MyDrive/BTech_Project/test_data_normal.pkl", "rb")
(og_generated_sentences, ft_generated_sentences, reference_sentences, keyword_list) = pickle.load(f)

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
sf = SmoothingFunction()
bleu_score = []
for i,j in zip(og_generated_sentences,reference_sentences):
  bleu_score.append(sentence_bleu(j,i,weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=sf.method2))
print(sum(bleu_score)/len(bleu_score))

0.07478386072325602


0.05266202270188934
